# TCP Server
Source - python Docs [socketserver](https://docs.python.org/3/library/socketserver.html)

In [2]:
import socketserver
import time, random, threading
import numpy as np
import rlgw
from rlgw.model import WORLD, ENV

  # Now use the WORLD object to create an ENV object 
Environment = ENV(  world = WORLD.get_world(
                  R=5, C=7, 
                  SP=(0,0), EP=(4,6), 
                  RNDSP=True, RNDEP=False, 
                  EMPTY=-1, RRN=[-5,-5], RRP=[5,5],  
                  RD=0.4, PRD=0.5, ER=0, 
                  MAXTS=5*7*2, 
                  SID=None, 
                  name="some.world"),
          name="some.env", 
          penalty=(False, 0),  #<--- if True, imposes penalty on agent for staying in the same cell
          always_reinit=False)#<--- if True, always choose a new random initial state from distribution

class game_gen:
    def __init__(self, env):
        self.env=env
        self.reset()
        
    def reset(self):
        self.env.reset()
        self.done=False
        self.ts=0

        
    def tocsv(self,npA, sep=','):
        res=""
        for i in range(len(npA)):
            res+=str(npA[i])+sep
        return res
    
    def prepare_bytes(self, time_steps):
        sep=','
        for i in range(time_steps):
            if self.done:
                self.reset()
            
            cS = np.copy(self.env.S)
            act = self.env.random_action()
            nS, Rew, self.done, _ = self.env.step(act)
            self.ts+=1
            res = str(self.ts) +sep+ str(int(self.done))+sep+ str(act) + sep+ \
                    str(Rew) +sep+ str(len(cS)) +sep+self.tocsv(cS, sep=sep) + self.tocsv(nS, sep=sep)
            # timestep, done, action, reward, state_len, cS, nS

        return bytes(res, encoding='utf-8')



# start server

In [3]:
games = game_gen(Environment)

#<---- as found on python Docs socketserver
HOST, PORT = "localhost", 9998
class SERVER(socketserver.BaseRequestHandler):       
    def handle(self):
        """
        In addition to the literal forms, bytes objects can be created in a number of other ways:
            A zero-filled bytes object of a specified length: bytes(10)
            From an iterable of integers: bytes(range(20))
            Copying existing binary data via the buffer protocol: bytes(obj)
        """
        global games
        data_bytes = games.prepare_bytes(1)# --> Note: finally data is converted to bytes
        print('Sending:', len(data_bytes), ' bytes')
        self.request.sendall(data_bytes)
        time.sleep(0.1) #<------------------ sleep time b/w handle
        

with socketserver.TCPServer((HOST, PORT), SERVER) as server:
    print('Started!')
    server.serve_forever() # interrupt the program with Ctrl-C
    
        

Started!
Sending: 228  bytes
Sending: 228  bytes
Sending: 228  bytes
Sending: 228  bytes
Sending: 228  bytes
Sending: 228  bytes
Sending: 228  bytes
Sending: 228  bytes
Sending: 228  bytes
Sending: 229  bytes
Sending: 229  bytes
Sending: 229  bytes
Sending: 229  bytes
Sending: 229  bytes
Sending: 229  bytes
Sending: 229  bytes
Sending: 229  bytes
Sending: 231  bytes
Sending: 231  bytes
Sending: 231  bytes
Sending: 231  bytes
Sending: 231  bytes
Sending: 231  bytes
Sending: 231  bytes
Sending: 231  bytes
Sending: 231  bytes
Sending: 231  bytes
Sending: 231  bytes
Sending: 231  bytes
Sending: 231  bytes
Sending: 231  bytes
Sending: 233  bytes
Sending: 233  bytes
Sending: 233  bytes
Sending: 233  bytes
Sending: 233  bytes
Sending: 233  bytes
Sending: 233  bytes


KeyboardInterrupt: 